In [1]:
from pyhdf.SD import SD, SDC
import numpy as np
import matplotlib.pyplot as plt
import pprint
from os import listdir
import time
import h5py
import pandas as pd
from datetime import datetime, date, timedelta
import os
import sys
from scipy import stats

In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [3]:
def write_file(n_array, starttime, endtime):
    fpath = "D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/reduced"
    os.chdir(fpath)
    year = endtime.strftime('%Y')
    path = '-'.join((starttime.strftime('%m%d'), endtime.strftime('%m%d')))
    if not os.path.exists(year):#this is the first file being created for that time
        os.makedirs(year)
        #set index to 0

    path = '/'.join((year,path))
    path = '.'.join((path,"h5"))
    print path
    with h5py.File(str(path).replace("pkl","h5"), mode="w") as f:
        f.create_dataset("data", data=n_array.to_records(index=False))

In [20]:
file_name = 'D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/cloud/2018/summer/MYD06_L2.A2018152.0020.061.2018152170514.hdf'
file = SD(file_name, SDC.READ)

In [21]:
datasets_dic = file.datasets()

for idx,sds in enumerate(datasets_dic.keys()):
    print idx,sds

0 Cloud_Top_Height_Nadir_Night
1 Single_Scatter_Albedo_Ice
2 Radiance_Variance
3 Brightness_Temperature
4 Sensor_Azimuth
5 Statistics_1km_sds
6 Cloud_Water_Path_1621
7 Cloud_Phase_Infrared_Day
8 Cloud_Effective_Radius_16_PCL
9 Sensor_Zenith
10 Cloud_Water_Path_1621_PCL
11 cloud_top_method_1km
12 Tropopause_Height
13 Cloud_Water_Path
14 Cloud_Top_Pressure_Nadir_Day
15 Longitude
16 IRW_Low_Cloud_Temperature_From_COP
17 Cloud_Fraction_Nadir
18 Cloud_Top_Pressure_From_Ratios
19 Atm_Corr_Refl
20 Cirrus_Reflectance
21 Cloud_Top_Height_Nadir_Day
22 Cloud_Optical_Thickness_37_PCL
23 Cloud_Optical_Thickness_Uncertainty
24 Cloud_Effective_Radius
25 Cloud_Effective_Radius_PCL
26 Cloud_Fraction_Day
27 Cloud_Top_Pressure_Day
28 Cloud_Optical_Thickness_16_PCL
29 Cloud_Effective_Radius_1621_PCL
30 Solar_Zenith
31 cloud_emiss12_1km
32 Sensor_Azimuth_Day
33 Latitude
34 cloud_emissivity_1km
35 Cloud_Top_Temperature_Night
36 Sensor_Zenith_Night
37 Scan_Start_Time
38 Cloud_Optical_Thickness_1621
39 Cloud_

In [22]:
lat_obj = file.select('Latitude') # select sds
lat = lat_obj.get()

lon_obj = file.select('Longitude') # select sds
lon = lon_obj.get()

data_obj = file.select('Cloud_Fraction') # select sds
data = data_obj.get()/100.

starttime_obj = file.select('Scan_Start_Time') # select sds
starttime = starttime_obj.get()

brightness_temp_obj = file.select('Brightness_Temperature')
brightness_temp = brightness_temp_obj.get()

cloud_optical_thickness_obj = file.select('Cloud_Optical_Thickness')
cloud_optical_thickness = cloud_optical_thickness_obj.get()

quality_assurance_obj = file.select('Quality_Assurance_5km')
quality_assurance = quality_assurance_obj.get()

cloud_water_path_obj = file.select('Cloud_Water_Path')
cloud_water_path = cloud_water_path_obj.get()


In [23]:
pprint.pprint( data_obj.attributes() )

{'Cell_Across_Swath_Sampling': [3, 1348, 5],
 'Cell_Along_Swath_Sampling': [3, 2028, 5],
 'Geolocation_Pointer': 'Internal geolocation arrays',
 'Parameter_Type': 'Output',
 '_FillValue': 127,
 'add_offset': 0.0,
 'long_name': 'Cloud Fraction in Retrieval Region (5x5 1-km Pixels) from 1-km Cloud Mask',
 'scale_factor': 0.009999999776482582,
 'units': 'none',
 'valid_range': [0, 100]}


In [24]:
plt.figure()
plt.plot(data)

In [25]:
lonmin = lon.min()
lonmax = lon.max()
latmin = lat.min()
latmax = lat.max()
print lonmin, lonmax
print latmin, latmax
print len(lon), len(lat)

-179.70644 179.83208
65.42532 89.933624
406 406


In [26]:
delta = 0.1
rows, cols = np.where((abs(lat - 72.) <= delta) & (abs(lon - -38) <= delta))

In [27]:
latsub = lat[rows][:,cols]
lonsub = lon[rows][:,cols]
datasub = data[rows][:,cols]
#qualitysub = quality[rows][:,cols]

In [28]:
plt.figure()
plt.pcolormesh(lon, lat, data, cmap='jet')
plt.colorbar(label = 'cloud fraction')
plt.clim(0.,datasub.max())
#plt.axis([lonmin, lonmax, latmin, latmax])
plt.ylabel('latitude (deg)')
plt.xlabel('longitude (deg)')
#plt.title('%s %s data binned to azimuth and revolution #' % (name, var))
plt.grid()
plt.show()

ValueError: zero-size array to reduction operation maximum which has no identity

In [13]:
latstepdeg = 1.
longsetdeg = 1.
startday = 1
scalefactor = 100.

In [14]:
summit_lon = -38.4592
summit_lat = 72.5796

In [15]:
y = 2018
season = 'summer'
directory = 'D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/%d/%s/' % (y, season)
files = listdir(directory)

pwv = []
TIME = []
quality_assurance = []
isummit_shape = []
numfiles = 0
cloudy_fraction = []
clear_fraction = []
latitude = []
longitude = []

print 'total number of files: %d' % len(files)
t1 = time.time()
for f in files:
    
    numfiles += 1
    
    try:
        file = SD(directory+f, SDC.READ)
    except:
        print 'passed: %s' % f
        continue
        
    lat_obj = file.select('Latitude') # select sds
    lat = lat_obj.get()

    lon_obj = file.select('Longitude') # select sds
    lon = lon_obj.get()
    
    delta = 0.05
    rows, cols = np.where((abs(lat - summit_lat) <= delta) & (abs(lon - summit_lon) <= delta))

    if len(rows) == 0 or len(cols) == 0:
        continue
        #print '# files: %d, ' % numfiles, f

    data_obj = file.select('Water_Vapor_Infrared') # select sds
    data = data_obj.get()/100.

    starttime_obj = file.select('Scan_Start_Time') # select sds
    starttime = starttime_obj.get()

    quality_obj = file.select('Quality_Assurance_Near_Infrared')
    quality = quality_obj.get()
    
    datasub = data[rows][:,cols]
    timesub = starttime[rows][:,cols]
    qualitysub = quality[rows][:,cols]
    lonsub = lon[rows][:,cols]
    latsub = lat[rows][:,cols]
    
    
    pwv.append(datasub[datasub>=0.].mean())
    TIME.append(timesub.mean())
    #quality_assurance.append(qualitysub)
    clear = float(len(datasub[datasub>=0]))
    cloudy = float(len(datasub[datasub<0]))
    latitude.append(latsub)
    longitude.append(lonsub)
    
    
    if clear == 0:
        cloudy_fraction.append(1.)
        clear_fraction.append(0.)
    elif cloudy == 0:
        clear_fraction.append(1.)
        cloudy_fraction.append(0.)
    else:
        cloudy_fraction.append(cloudy/(clear+cloudy))
        clear_fraction.append(clear/(clear+cloudy))
        
    sys.stdout.write("\rPercent complete: %.2f, time elapsed: %.2f min" % ((float(numfiles) * 100. / float(len(files))), (time.time()-t1)/60.))
    sys.stdout.flush()
        
t2 = time.time()
print t2-t1
pwv = np.asarray(pwv)
TIME = np.asarray(TIME)
cloudy_fraction = np.asarray(cloudy_fraction)
clear_fraction = np.asarray(clear_fraction)
latitude = np.asarray(latitude)
longitude = np.asarray(longitude)
#quality_assurance = np.asarray(quality_assurance)

WindowsError: [Error 3] The system cannot find the path specified: 'D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/2018/summer/*.*'

In [16]:
lat_diff = []
for i in range(len(latitude)):
    if len(latitude[i]) > 1:
        for j in range(len(latitude[i])):
            if len(latitude[i][j]) > 1:
                for k in range(len(latitude[i][j])):
                    lat_diff.append(latitude[i][j][k] - summit_lat)
            else:
                lat_diff.append(latitude[i][j] - summit_lat)
    else:
        lat_diff.append(latitude[i] - summit_lat)
        
lat_diff = np.asarray(lat_diff)

NameError: name 'latitude' is not defined

In [17]:
print lat_diff[1]

IndexError: list index out of range

In [ ]:
plt.figure()
plt.plot(lat_diff)
plt.ylabel('lat diff')
plt.title('max diff = %.2f, min diff = %.2f' % (lat_diff.max(), lat_diff.min()))

In [ ]:
plt.figure()
plt.plot(TIME, cloudy_fraction)

In [ ]:
doysecoffset= 23*60*60*24*365 + 7*60*60*24 - 17*60*60 - 60 - 5.761243
tt = TIME + doysecoffset

In [ ]:
h5data = pd.DataFrame({"pwv" : pwv})
h5data["cloudy_fraction"] = cloudy_fraction
h5data["time"] = tt 

In [ ]:
starttime = datetime.fromtimestamp(tt[0])
endtime = datetime.fromtimestamp(tt[-1])
print starttime
print endtime

In [ ]:
write_file(h5data, starttime, endtime)

In [4]:
TIME = {}
pwv = {}
cloudy_fraction = {}
year = [2013, 2014, 2015, 2016, 2017, 2018]

for y in year:
    fpath = "D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/pwv/reduced/%s/" % y
    files = listdir(fpath)
    TIME[y] = {}
    pwv[y] = {}
    cloudy_fraction[y] = {}
    for f in files:
        dset = h5py.File(fpath+f, 'r')['data']
        
        if f[:2] == '12':
            season = 'winter'
        else:
            season = 'summer'
            
        cloudy_fraction[y][season] = dset['cloudy_fraction']
        TIME[y][season] = dset['time']
        pwv[y][season] = dset['pwv']

In [5]:
print cloudy_fraction[2018]['summer'].shape
print pwv[2018]['summer'].shape

(430L,)
(430L,)


In [66]:
plt.figure()
plt.plot(pwvmean[2017]['summer']/10., label = 'summer')
plt.plot(pwvmean[2017]['winter']/10., label = 'winter')
plt.xlabel('samples')
plt.ylabel('pwv (cm)')
plt.title('Summit 2017 daily averages')
plt.legend()

KeyError: 2017

In [33]:
timespan = 3600.*24*30

In [34]:
cloudmean = {}
tcloudmean = {}
cloudstd = {}
season = ['summer', 'winter']
i0=0
for y in year:
    tcloudmean[y] = {}
    cloudstd[y] = {}
    cloudmean[y] = {}
    for s in season:
        cf = cloudy_fraction[y][s][np.logical_not(np.isnan(cloudy_fraction[y][s]))]
        ti = TIME[y][s][np.logical_not(np.isnan(TIME[y][s]))]
        cmean = []
        smean = []
        tmean = []
        t = TIME[y][s]
        for h in range(len(t)):
            if t[h] - t[i0] >= timespan:
                cmean.append(cf[i0:h+1].mean())
                smean.append(cf[i0:h+1].std())
                tmean.append(ti[i0:h+1].mean())
                i0=h+1
            if i0<len(t):
                cmean.append(cf[i0:len(t)+1].mean())
                tmean.append(ti[i0:len(t)+1].mean())
                smean.append(cf[i0:len(t)+1].std())
        i0 = 0
        cloudmean[y][s] = np.asarray(cmean)
        cloudstd[y][s] = np.asarray(smean)
        tcloudmean[y][s] = np.asarray(tmean)

In [7]:
print datetime.fromtimestamp(tcloudmean[2017]['winter'][-1])

2017-02-28 14:16:55.997241


In [8]:
for y in year:
    for s in season:
        print y, s
        print len(pwvmean[y][s])
        print len(tcloudmean[y][s])
        print len(cloudmean[y][s])
        print '---------------'

2013 summer


NameError: name 'pwvmean' is not defined

In [36]:
scale = 1.
bins=50
plt.figure()
i = 0
plt.suptitle('Summit cloudy fraction daily averages, Prob(cloudy fraction $>$ x)')

for y in year:
    
    plt.subplot(3, 2, i+1)
    i += 1
    for s in season:
        print y, s
        values, base = np.histogram(cloudmean[y][s], bins=bins, density=True)
        cumulative = np.cumsum(values)
        cumu = cumulative/cumulative.max()
        cumu_I = 1-cumu
        if s == 'winter':
            plt.plot(base[:-1]*scale, cumu_I*100, '--', linewidth = 2, label = ' %s %d' % (s, y))
        else:
            plt.plot(base[:-1]*scale, cumu_I*100, linewidth = 2, label = ' %s %d' % (s, y))
    
    if y >= 2017:
        plt.xlabel('cloudy sky fraction')
    if (y % 2) == 1:
        plt.ylabel('Cumulative Distribution (%)')
    #plt.ylim([0, 80])
    plt.xlim([0, 1.1])
    plt.legend()
    plt.grid()
plt.show()

2013 summer
2013 winter
2014 summer
2014 winter
2015 summer
2015 winter
2016 summer
2016 winter
2017 summer
2017 winter
2018 summer
2018 winter


In [9]:
print season

['summer', 'winter']


In [10]:
scale = 1.
bins=50
plt.figure()
i = 0
plt.suptitle('Summit PWV daily averages, Prob(cloudy fraction $>$ x)')

for y in year:
    
    plt.subplot(3, 2, i+1)
    i += 1
    for s in season:
        print y, s
        values, base = np.histogram(pwvmean[y][s], bins=bins, density=True)
        cumulative = np.cumsum(values)
        cumu = cumulative/cumulative.max()
        cumu_I = 1. - cumu
        if s == 'winter':
            plt.plot(base[:-1]*scale, cumu_I*100, '--', linewidth = 2, label = ' %s %d' % (s, y))
        else:
            plt.plot(base[:-1]*scale, cumu_I*100, linewidth = 2, label = ' %s %d' % (s, y))
    
    if y >= 2017:
        plt.xlabel('PWV (mm)')
    if (y % 2) == 1:
        plt.ylabel('Cumulative Distribution (%)')
    #plt.ylim([0, 80])
    plt.xlim([0, 1.1])
    plt.legend()
    plt.grid()
plt.show()

2013 summer


NameError: name 'pwvmean' is not defined

In [ ]:
print pwvmean[2018]['summer']
print pwvmean[2018]['winter'].shape

In [11]:
#y = 2018
bins = 50
minimum = 0.5
maximum = 1
plt.figure()
for i in range(len(year)):
    plt.subplot(3, 2, i+1)
    s = 'summer'
    plt.hist(cloudmean[year[i]][s], bins = bins, range = (minimum, maximum), label='%s %d' % (s, year[i])) 
    s = 'winter'
    plt.hist(cloudmean[year[i]][s], bins = bins, range = (minimum, maximum), label='%s %d' % (s, year[i])) 
    #if i >= 2:
    plt.xlabel('cloudy sky fraction')
    #if i == 0 or i == 2:
    plt.ylabel('counts')
    plt.legend()
    plt.grid()

In [12]:
def nummonth(name):
    if name == 'Jun':
        num = 6
    elif name == 'Jul':
        num = 7
    elif name == 'Aug':
        num = 8
    else:
        print(name)
    return num

In [13]:
def get_date_indices(d, tdata, data, days):
    
    yr = int('%s' % d[-4:])
    
    date = datetime.strptime(d, "%m/%d/%Y")
    time_tuple = date.timetuple()
    tstamp1 = time.mktime(time_tuple)
    
    tstamp2 = tstamp1 + days*24*60*60

    itime = np.where((tdata >= tstamp1) & (tdata < tstamp2))
    data = data[itime]
    tdata = tdata[itime]
    
    datasort = np.argsort(tdata)
    
    data = data[datasort]
    tdata = tdata[datasort]

        
    return data, tdata

In [14]:
#read data from txt file
tskyfrac = {}
skyfrac = {}
YEAR = [2013, 2014, 2015, 2016, 2017, 2018]
for y in YEAR:
    print(y)
    t = []
    csky = []
    with open('D:/software_git_repos/polaris/polaris_data/weather_data/TSI_data/sky_fraction/%d.txt' % y, 'r') as inputfile:
        next(inputfile)
        for line in inputfile:
            l = line.split(',')
            try:
                t.append(l[0])
                csky.append(float(l[1]))
            except:
                pass
    tskyfrac[y] = t
    skyfrac[y] = np.asarray(csky)
    
    d = []
    for t in range(len(tskyfrac[y])):
        month = nummonth(tskyfrac[y][t][6:9])
        day = tskyfrac[y][t][10:12]
        hour = tskyfrac[y][t][13:15]
        minute = tskyfrac[y][t][16:18]
        second = tskyfrac[y][t][19:21]
    
        date = datetime(y, month, int(day), int(hour), int(minute), int(second))
        time_tuple = date.timetuple()
        d.append(time.mktime(time_tuple))
        
    tskyfrac[y] = np.asarray(d)

2013
2014
2015
2016
2017
2018


In [15]:
TIME = {}
pwv = {}
cloudy_fraction = {}
year = [2013, 2014, 2015, 2016, 2017, 2018]

for y in year:
    fpath = "D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/pwv/reduced/%s/" % y
    files = listdir(fpath)
    TIME[y] = {}
    pwv[y] = {}
    cloudy_fraction[y] = {}
    for f in files:
        dset = h5py.File(fpath+f, 'r')['data']
        
        if f[:2] == '12':
            season = 'winter'
        else:
            season = 'summer'
            
        cloudy_fraction[y][season] = dset['cloudy_fraction']
        TIME[y][season] = dset['time']
        pwv[y][season] = dset['pwv']

In [16]:

sf = {}
tsf = {}

for y in YEAR:
    sf[y] = {}
    tsf[y] = {}
    dg = '06/01/'+str(y)
    season = 'summer'
    numdays = 30+2*31
    sf[y][season], tsf[y][season] = get_date_indices(dg, tskyfrac[y], skyfrac[y], (numdays))
    dg = '12/01/'+str(y-1)
    season = 'winter'
    sf[y][season], tsf[y][season] = get_date_indices(dg, tskyfrac[y], skyfrac[y], (numdays))

In [35]:
sfmean = {}
tsfmean = {}
sfstd = {}
season = ['winter', 'summer']
for y in YEAR:
    sfmean[y] = {}
    tsfmean[y] = {}
    sfstd[y] = {}
    for seas in season:
        sfm = []
        tsfm = []
        sfs = []
        i0 = 0
        for s in range(len(tsf[y][seas])):
            if tsf[y][seas][s]-tsf[y][seas][i0] >= timespan:
                sfm.append(sf[y][seas][i0:s+1].mean())
                tsfm.append(tsf[y][seas][i0:s+1].mean())
                sfs.append(sf[y][seas][i0:s+1].std())
                i0=s+1
        if i0<len(tsf[y]):
            sfm.append(sf[y][seas][i0:len(sf[y][seas])+1].mean())
            tsfm.append(tsf[y][seas][i0:len(tsf[y][seas])+1].mean())
            sfs.append(sf[y][seas][i0:len(sf[y][seas])+1].std())

        sfmean[y][seas] = np.asarray(sfm)
        tsfmean[y][seas] = np.asarray(tsfm)
        sfstd[y][seas] = np.asarray(sfs)

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice.
C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: Mean of empty slice.


In [36]:
p = {}
pstd = {}
sfsorted = {}
stat = {}
r = {}

for y in YEAR:
    p[y] = {}
    pstd[y] = {}
    sfsorted[y] = {}
    stat[y] = {}
    r[y] = {}
    rr = []
    for s in season:
        p[y][s] = np.interp(tsfmean[y][s], tcloudmean[y][s], cloudmean[y][s])
        pstd[y][s] = np.interp(tsfmean[y][s], tcloudmean[y][s], cloudstd[y][s])
        isort = np.argsort(sfmean[y][s])
        p[y][s] = p[y][s][isort]
        pstd[y][s] = pstd[y][s][isort]
        sfsorted[y][s] = sfmean[y][s][isort]
        stat[y][s] = stats.pearsonr(sfsorted[y][s], pstd[y][s])
        rr.append(stat[y][s][0])
        print y, stat[y][s], s
    r[y][s] = rr

2013 (nan, nan) winter
2013 (0.8065969107257464, 0.40261377176033397) summer
2014 (nan, nan) winter
2014 (0.8272743406706207, 0.3797804830593104) summer
2015 (nan, nan) winter
2015 (0.8857341044556508, 0.30731115280837634) summer
2016 (nan, nan) winter
2016 (-0.8631038950931109, 0.3370344997035992) summer
2017 (nan, nan) winter
2017 (0.9958968286472384, 0.057690401083054246) summer
2018 (nan, nan) winter
2018 (0.8664786776126105, 0.3327557532806502) summer


In [37]:
scale = 1.
bins=50
plt.figure()
i = 0
plt.suptitle('cloudy sky fraction daily averages')
s = 'summer'
for y in year:
    
    plt.subplot(3, 2, i+1)
    i += 1
        
    #plt.errorbar(sfsorted[y][s], p[y][s], yerr = pstd[y][s], capsize=2, fmt='.', label = '%d, r = %.2f, p = %.2E' % (y, stat[y][s][0], stat[y][s][1]))
    plt.plot(sfsorted[y][s], p[y][s], '-', label = '%d, r = %.2f, p = %.2E' % (y, stat[y][s][0], stat[y][s][1]))
    if y >= 2017:
        plt.xlabel('Ground measurement')
    if (y % 2) == 1:
        plt.ylabel('Sattelite measurement')
    #plt.ylim([0, 80])
    plt.xlim([0, 1.1])
    plt.legend()
    plt.grid()
plt.show()

In [38]:
print len(sfsorted[2018]['summer'])

3
